In [1]:
import numpy as np
import pandas as pd


In [2]:
data = open('./kafka.txt','r').read()

In [3]:
char_list = list(set(data))
data_size,vocabulary_size = len(list(data)),len(char_list)
print('Len of vocab :',vocabulary_size ,'len of data : ' ,data_size)

Len of vocab : 80 len of data :  137628


In [4]:
char_ix ={ch : i for i,ch in enumerate(char_list)}
ix_char ={i : ch for i,ch in enumerate(char_list)}

print(char_ix)
print(ix_char)

{'R': 0, 'U': 1, 's': 2, 'O': 3, '6': 4, 'z': 5, 'k': 6, 'p': 7, '@': 8, '8': 9, '\n': 10, 'd': 11, 'o': 12, 'Q': 13, 'h': 14, 'q': 15, 't': 16, '4': 17, 'T': 18, "'": 19, 'B': 20, '3': 21, 'm': 22, '!': 23, 'f': 24, 'r': 25, 'H': 26, '"': 27, 'Y': 28, 'F': 29, '(': 30, 'A': 31, '0': 32, 'g': 33, 'w': 34, '-': 35, ':': 36, 'K': 37, 'M': 38, 'e': 39, '*': 40, ' ': 41, '9': 42, 'i': 43, 'J': 44, 'ç': 45, 'j': 46, 'n': 47, '.': 48, '5': 49, 'C': 50, ')': 51, '7': 52, '$': 53, 'L': 54, 'b': 55, 'I': 56, 'S': 57, 'P': 58, 'x': 59, 'a': 60, 'l': 61, 'E': 62, ',': 63, 'G': 64, 'y': 65, '?': 66, 'c': 67, 'D': 68, 'N': 69, ';': 70, '1': 71, '%': 72, 'V': 73, 'u': 74, 'W': 75, 'X': 76, '2': 77, '/': 78, 'v': 79}
{0: 'R', 1: 'U', 2: 's', 3: 'O', 4: '6', 5: 'z', 6: 'k', 7: 'p', 8: '@', 9: '8', 10: '\n', 11: 'd', 12: 'o', 13: 'Q', 14: 'h', 15: 'q', 16: 't', 17: '4', 18: 'T', 19: "'", 20: 'B', 21: '3', 22: 'm', 23: '!', 24: 'f', 25: 'r', 26: 'H', 27: '"', 28: 'Y', 29: 'F', 30: '(', 31: 'A', 32: '0',

In [5]:
#hyperparameters
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

In [7]:
#model prarameters
#input to hidden state
Wxh =np.random.randn(hidden_size,vocabulary_size)* 0.01


#hidden state to hidden state
Whh =np.random.randn(hidden_size,hidden_size)* 0.01
bh =np.zeros((hidden_size,1))

#hidden state to output state
Why =np.random.randn(vocabulary_size,hidden_size)* 0.01
bhy =np.zeros((vocabulary_size,1))

In [8]:
def loss_fun(inputs,target,hprev):
    
    xs,hs,ys,ps = {},{},{},{}
    
    hs[-1]= np.copy(hprev)
    loss = 0
    
    for t in range(len(inputs)):
        #create One hot vector 
        xs[t] = np.zeros((vocabulary_size,1))
        xs[t][inputs[t]] = 1
        
        #feed forward network
        hs[t] = np.tanh(np.dot(Wxh,xs[t])  + np.dot(Whh,hs[t-1]) + bh)
        
        ys[t] = np.dot(Why,hs[t]) + bhy
        
        #softmax layer
        ps[t] = np.exp(ys[t])/np.sum(np.exp(ys[t]))
        
        loss += -np.log(ps[t][target,0])#softmax cross entropy
        
    dWxh,dWhh,dWhy = np.zeros_like(Wxh),np.zeros_like(Whh),np.zeros_like(Why)
    dbh,dbhy = np.zeros_like(bh),np.zeros_like(bhy)
    dhnext = np.zeros_like(hs[0])
    
    
    for t in reversed(range(len(inputs))):
        
        dy = np.copy(ps[t])
        
        dy[target[t]] -= 1
        
        
        dWhy += np.dot(dy,hs[t].T)
        
        dbhy += dy
        
        
        dh = np.dot(Why.T,dy) +dhnext
        #backporp through tanh
        dhraw = (1 - hs[t]*hs[t]) *dh
        
        dbh += dhraw
        dWhh += np.dot(dhraw,hs[t-1].T)
        dWxh += np.dot(dhraw,xs[t].T)
        
        dhnext = np.dot(Whh.T,dhraw)
        
        
    #clip if gradient is exploding 
    for params in [dWxh,dWhh,dWhy,dbh,dbhy]:
        np.clip(params,-5,5,out=params)
        
    return loss,dWxh,dWhh,dWhy,dbh,dbhy,hs[len(inputs)-1]
            
        

In [30]:
def sampling(h,seed_ix,n):
    
    x = np.zeros((vocabulary_size,1))
    x[seed_ix]=1
    #store genrated char
    ixes =[]
    
    for t in range(n):
        h = np.tanh(np.dot(Wxh,x) + np.dot(Whh,h) + bh)
        
        y = np.dot(Why,h) + bhy
        
        p = np.exp(y)/np.sum(np.exp(y))
        
        ix = np.random.choice(range(vocabulary_size),p=p.ravel())
        
        ixes.append(ix)
        x =  np.zeros((vocabulary_size,1))
        x[ix]=1
        
    message = ''.join(ix_char[ix] for ix in ixes)
    
    print('\n------',message,'------\n')
    

In [31]:
hprev = np.zeros((hidden_size,1))
sampling(hprev,5,12)


------ e ree,, they ------



In [32]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(bhy) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/ vocabulary_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    # check "How to feed the loss function to see how this part works
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
        p = 0 # go from start of data                                                                                                                                                             
    inputs = [char_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # forward seq_length characters through the net and fetch gradient                                                                                                                          
    loss, dWxh, dWhh, dWhy, dbh, dbhy, hprev = loss_fun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # sample from the model now and then                                                                                                                                                        
    if n % 1000 == 0:
        print('iter ', n, 'loss:', smooth_loss) # print progress
        sampling(hprev, inputs[0], 50)

    # perform parameter update with Adagrad                                                                                                                                                     
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, bhy],
                                [dWxh, dWhh, dWhy, dbh, dbhy],
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

    p += seq_length # move data pointer                                                                                                                                                         
    n += 1 # iteration counter


iter  0 loss: [109.58748664 109.55167883 109.55034087 109.61918317 109.58968657
 109.58083934 109.58748664 109.5755428  109.58748664 109.6321103
 109.61614155 109.55034087 109.60965348 109.64007153 109.55167883
 109.58748664 109.55034087 109.72245672 109.58083934 109.55167883
 109.6321103  109.58968657 109.58083934 109.55034087 109.729654  ]

------ gank, y ruth in it the mears, stated his with then ------

iter  1000 loss: [104.36735393 103.78102342 103.96967796 102.72956651 103.99804021
 104.50257185 103.18842274 104.14345012 104.30962947 104.8433193
 104.33396284 104.35626437 104.37081377 103.84356963 104.75438769
 104.86324465 103.36482033 104.06210415 105.20719886 103.54143926
 104.51740323 103.43364433 104.54594015 105.48062048 104.40986478]

------  eay rous cfenad hapeiy rfede? deantd. "Librg whif ------

iter  2000 loss: [103.01572196 102.4867097  102.62523881 101.81190608 103.77347436
 101.82473849 102.61880467 102.90396706 103.17015792 102.85867533
 102.97199157 102.82794382

iter  20000 loss: [114.43077052 115.10021706 114.45815445 114.91940886 116.03797642
 114.53345218 116.23816413 114.15164844 115.36412588 116.59624013
 114.12577388 116.76181235 115.74612804 115.14105494 114.06262681
 115.91716563 114.70899549 115.1172818  114.70154588 115.00741755
 114.58268729 115.7466073  115.28329279 114.27117538 113.98934107]

------ rlle wornight it sang inpsish halle, and or aine t ------

iter  21000 loss: [115.67388858 116.03748767 114.91507761 117.12126777 115.25234361
 115.9515246  117.6677251  114.95144359 117.67861572 116.34792634
 115.25580223 117.24715022 116.92685102 116.77508257 116.12889672
 116.40985699 116.32888469 115.91595674 116.62015862 116.49545313
 116.03113929 116.90279874 115.57502286 116.47659564 115.07812868]

------ ver. "Wifup, anhe bashed ack to an fathying, bymer ------

iter  22000 loss: [115.75533246 116.96671978 116.83856254 116.56420356 116.97460067
 117.1710258  116.51325757 116.60624784 117.72982212 116.08446175
 115.69295522 117.

iter  40000 loss: [119.34067322 119.08127083 120.78223828 119.51215822 120.6646604
 118.73301746 119.67806206 121.01862157 119.64661951 120.17360423
 120.58184013 120.53623807 119.66178724 119.81933895 119.63675495
 120.1067076  119.43250988 120.18010819 120.62772697 119.51534332
 119.52424903 118.7527938  121.1581196  120.58486197 119.56672718]

------ inDing and forgcesthing the tliting she chacll uue ------

iter  41000 loss: [119.20257248 119.48015911 119.22884405 119.65597287 120.42682856
 118.91052427 120.67592741 118.90598617 120.36672002 120.42048455
 119.60612104 120.82620238 119.65635072 120.05853089 118.4596798
 120.10705552 119.39620583 120.03276867 119.59133244 118.95475488
 119.31978217 119.6104958  120.67917949 120.07500488 118.33169108]

------ wint Onter lithit for Gredor of in disinte read ev ------

iter  42000 loss: [119.80899552 120.45361242 119.85298937 120.12507349 121.1349671
 120.07055123 121.48213366 119.42219578 120.444375   121.80417731
 119.47728997 122.127

iter  60000 loss: [123.62315257 124.33959989 124.18625426 124.53422722 123.80401863
 124.23381037 126.23789928 124.2916861  126.77228147 124.77567041
 123.52543156 125.83519945 124.33269466 124.61552037 125.03189229
 124.81310747 124.50109442 124.65328699 124.80450003 124.64797126
 123.95146183 126.36325743 123.82513149 124.87168149 124.45353762]

------ kly a come as ho stup hans being distcemtlet lande ------

iter  61000 loss: [122.46204126 123.84697179 124.07560137 123.56764369 125.04810432
 123.98026423 123.65780174 124.4447342  124.63202833 123.79800417
 122.55572613 124.26551425 122.84793051 123.91999657 123.07952927
 122.83081933 123.22754373 123.5334508  124.41609705 124.63085502
 122.83483406 124.08110137 122.81053592 123.71637904 124.17108175]

------ , the dingafinemenire and at to says, was he in to ------

iter  62000 loss: [123.15849964 122.89787574 124.52738175 123.25235829 124.20401275
 122.66993081 123.3514401  124.96774551 123.50156107 123.69522816
 124.09584532 124.

iter  80000 loss: [124.66035391 125.20833173 125.22009729 125.91809696 126.6964945
 124.65523461 126.64414384 124.7688144  126.042568   127.41961962
 125.50045405 126.16874213 125.89658224 126.19062504 124.93393235
 126.46905232 124.79068945 125.34341065 125.12807023 124.16677854
 125.61802596 125.06557746 126.91633594 125.06639131 124.77007486]

------ : TULT But theim, betpere unance sees had and; as  ------

iter  81000 loss: [125.27473337 125.85611161 125.98492646 125.55168371 125.1678049
 125.74342993 126.71398794 125.31147607 126.48757427 127.86459828
 125.81178735 127.15249432 126.11580596 126.85136481 125.8397547
 126.07004362 125.46522067 126.21405308 126.02102251 125.8812762
 125.64738149 126.58716409 125.72328005 125.64626765 123.98834531]

------ es", lon
sfistour at whowe ar door;.

Iffed him so ------

iter  82000 loss: [125.95433215 126.85532016 126.53679694 127.06631122 126.11744461
 126.84572837 129.07427289 126.55371463 129.37645267 127.1430293
 125.85354052 128.41044

iter  100000 loss: [127.30631966 127.34051955 128.47364398 127.58809082 128.010711
 128.89646979 126.80597604 128.39253568 129.16485798 128.48010372
 126.97712189 128.3482179  127.59172705 127.90823564 128.65346565
 129.22079084 127.0303116  128.22284734 129.16056227 127.69398723
 127.49791862 127.8721324  128.11592112 129.0702389  127.85165491]

------ ny Gregor's upsaicalse ttrarses -  ing and sisitfi ------

